In [ ]:
import json
import torch

from tqdm.autonotebook import tqdm
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
model_kwargs = {
    "torch_dtype": torch.float16,
    "quantization_config": {"load_in_4bit": True},
    "low_cpu_mem_usage": True,
    "device_map": "auto",
}

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    **model_kwargs
)

In [ ]:
TRAIN_JSON = "/home/cs601-rnanawa1/llm-prompt-recovery/dataset/rewritten_text/artifacts/mini-dataset/train.json"
TEST_JSON = "/home/cs601-rnanawa1/llm-prompt-recovery/dataset/rewritten_text/artifacts/mini-dataset/test.json"

In [ ]:
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

In [ ]:
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))
